In [0]:
import numpy as np
import pandas as pd
import math
import re

In [0]:
#http://jusyo.jp/csv/new.phpからデータをダウンロード

!wget http://jusyo.jp/downloads/new/csv/csv_zenkoku.zip
!unzip csv_zenkoku.zip

In [0]:
filename = "zenkoku.csv"
data = pd.read_csv(filename, encoding="cp932")

In [0]:
key_data = data[["都道府県","市区町村","町域","町域補足","京都通り名","字丁目"]]

In [7]:
key_data.head()

,都道府県,市区町村,町域,町域補足,京都通り名,字丁目
0,北海道,札幌市中央区,NaN,（該当なし）,NaN,NaN
1,北海道,札幌市中央区,NaN,（該当なし）,NaN,NaN
2,北海道,札幌市中央区,NaN,（該当なし）,NaN,NaN
3,北海道,札幌市中央区,NaN,（該当なし）,NaN,NaN
4,北海道,札幌市中央区,NaN,（該当なし）,NaN,NaN


In [8]:
key_data.describe()

,都道府県,市区町村,町域,町域補足,京都通り名,字丁目
count,149415,149415,147213,2251,1135,6895
unique,47,1934,81062,7,1019,5058
top,北海道,港区,西新宿,（該当なし）,上の下立売通御前西入上る,２丁目
freq,10295,1394,631,2122,3,104


In [0]:
address_list=[]

for r in range(key_data.shape[0]):
    d = key_data.iloc[r,:]

    for i in range(len(d)):
        if d[i]!=d[i] or d[i]=='（該当なし）':
            d[i]=""
    full="".join(d)
    address_list.append(full)

In [0]:
unique_address_list = sorted(set(address_list), key=address_list.index)

In [11]:
print(len(unique_address_list))

126774


In [12]:
unique_address_list[:10]

['北海道札幌市中央区',
 '北海道札幌市中央区旭ケ丘',
 '北海道札幌市中央区大通西１丁目',
 '北海道札幌市中央区大通西２丁目',
 '北海道札幌市中央区大通西３丁目',
 '北海道札幌市中央区大通西４丁目',
 '北海道札幌市中央区大通西５丁目',
 '北海道札幌市中央区大通西６丁目',
 '北海道札幌市中央区大通西７丁目',
 '北海道札幌市中央区大通西８丁目']

In [0]:
#住所ファイルの保存
import pickle
with open('unique_address_list.pickle', 'wb') as f:
    pickle.dump(unique_address_list, f)

In [0]:
#住所ファイルの読み込み
import pickle
with open('unique_address_list.pickle', 'rb') as f:
    unique_address_list = pickle.load(f)

In [0]:
def address_check(text):
    search_key = "^.*" + ".*".join(text)
    sikutchoson = re.findall(r"[^都道府県市区町村]+[市|区|町|村]", text)
    if sikutchoson==[]:
        sikutchoson=[text]

    result = [t for t in unique_address_list if re.match(search_key, t)]
    if len(result)==0:
        return "error", ""
    elif len(result)==1:
        return "ok", result[0]
    else:
        result_judge = [sum([s in res for s in sikutchoson])==len(sikutchoson) for res in result]
        result2 = np.array(result)[result_judge]
        if len(result2)==1:
            return "ok", result2[0]
        else:
            return "not identify", result2

def check(text):
    n=len(text)
    for i in range(len(text)):
        code, c = address_check(text[:(n-i)])
        if code=="error":
            continue
        else:
            return c

In [16]:
print(check("中央区"))

['北海道札幌市中央区' '北海道札幌市中央区旭ケ丘' '北海道札幌市中央区大通西１丁目' ... '熊本県熊本市中央区世安町'
 '熊本県熊本市中央区万町' '熊本県熊本市中央区練兵町']


In [19]:
print(check("京都市東出町99-99"))

京都府京都市山科区勧修寺東出町
